# Response Time Simulation


More info on OSMNX:
Get street networks anywhere in the world from OpenStreetMap data then analyze and visualize them.

  - [Documentation and install instructions](https://osmnx.readthedocs.io)
  - [Examples, demos, tutorials](https://github.com/gboeing/osmnx-examples)
  - [Journal article and citation info](http://geoffboeing.com/publications/osmnx-complex-street-networks/)

In [1]:
import networkx as nx
import numpy as np
import geopandas as gpd
import fiona
import osmnx as ox
import requests
import skmob
import pandas as pd
import json
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import fiona
import warnings
from shapely.geometry import shape,mapping, Point, Polygon, MultiPolygon
warnings.filterwarnings("ignore")
import matplotlib.cm as cm
import matplotlib.colors as colors
import statistics
import os

import simulation_functions as sim

ox.config(use_cache=True, log_console=True)
ox.__version__

'1.1.1'

In [2]:
place = 'Chicago, Illinois, USA'
G = ox.graph_from_place(place, network_type='drive')
#fig, ax = ox.plot_graph(G)

stats documentation: https://osmnx.readthedocs.io/en/stable/osmnx.html#module-osmnx.stats

In [3]:
# assign speeds and traversal times to the edges
for u, v, k, d in G.edges(keys=True, data=True):
    if 'residential' in d['highway']:
        d['speed'] = 20
    elif 'tertiary' in d['highway']:
        d['speed'] = 30
    elif 'secondary' in d['highway']:
        d['speed'] = 40
    else:
        d['speed'] = 25
    
    # calculate time to traverse edge, in minutes
    d['time'] = d['length'] / (d['speed'] * 1609.34 / 60) #miles/hour to meters/minute

## Run Scenarios

In [4]:
#percent_operating_list=[0.04,0.06,0.08,0.10,0.12,0.14,0.16,0.18,0.20] 
#percent_operating_list=[0.10, 0.20, 0.30, 0.40, 0.50, 0.60, 0.70, 0.80, 0.90] 
percent_operating_list=[0.10,0.20,0.30,0.40,0.50,0.60,0.70,0.80,0.90]
                        
data_dict={}

def run_daily_profile(csv,grouping,percent_operating_list):
        #this function runs for different staffing scenarios
        
    crime_data,polrep_crime_data,altrep_crime_data=sim.create_crime_dfs(csv,grouping)
    PS_data,cr_lat,cr_long=sim.create_police_station_df(csv='Police_Stations_-_Map.csv',folder='Simulation_Data')
    Gs = ox.utils_graph.get_largest_component(G, strongly=True)
    PC_node,Police_officers=sim.create_pc_node(PS_data,G,Gs,cr_lat,cr_long)
    polrep_crime_node,polrep_crime_geo=sim.get_nearest_crime_node(polrep_crime_data,Gs)
    altrep_crime_node,altrep_crime_geo=sim.get_nearest_crime_node(altrep_crime_data,Gs)
    polrep_time_min,polrep_ser_min,polnum_units=sim.create_time_sequence(polrep_crime_data)
    polrep_Crime = [polrep_time_min,polrep_crime_node,polrep_ser_min,polrep_crime_geo,polnum_units]
    altrep_time_min,altrep_ser_min,altnum_units=sim.create_time_sequence(altrep_crime_data)
    alt_rep_Crime = [altrep_time_min,altrep_crime_node,altrep_ser_min,altrep_crime_geo,altnum_units]
    
    pol_demand=pd.DataFrame({'time':polrep_time_min,'num_units':polnum_units}).groupby('time').sum()
    pol_demand_tot={'Time(min)':pol_demand.index.tolist(),'#units':pol_demand['num_units'].tolist()}
                    
    alt_demand=pd.DataFrame({'time':altrep_time_min,'num_units':altnum_units}).groupby('time').sum()
    alt_demand_tot={'Time(min)':alt_demand.index.tolist(),'#units':pol_demand['num_units'].tolist()}
    
    polys=sim.create_polys(PS_data,Gs, shp='Chicago_PB.shp',folder='Simulation_Data')
    ndk=sim.set_ndk(Gs,PC_node)
    
    for percent_operating in percent_operating_list:
        polrep_RT,polrep_Officers_maxed_out,officer_demand_district=sim.eva_resp_time(PC_node,polrep_Crime,percent_operating,ndk,polys,Gs)  
        pol_res_time = sim.column(polrep_RT, 0)
        data_dict[str(grouping)+'_'+csv[:-13]+'_'+str(percent_operating)]=[np.mean(pol_res_time),np.median(pol_res_time),np.max(pol_res_time),polrep_Officers_maxed_out,pol_res_time,pol_demand_tot,officer_demand_district]
        
        percent_responding=round(1-percent_operating,2)
        altrep_RT,altrep_Officers_maxed_out,officer_demand_district=sim.eva_resp_time(PC_node,alt_rep_Crime,percent_responding,ndk,polys,Gs)
        alt_res_time = sim.column(altrep_RT, 0)
        data_dict['non'+str(grouping)+'_'+csv[:-13]+'_'+str(percent_responding)]=[np.mean(alt_res_time),np.median(alt_res_time),np.max(alt_res_time),altrep_Officers_maxed_out,alt_res_time,alt_demand_tot,officer_demand_district]
       



In [5]:
def run_BAU(csv,percent_operating_list):
        
    crime_data,gr1,gr2=sim.create_crime_dfs(csv,grouping='BAU')
    PS_data,cr_lat,cr_long=sim.create_police_station_df(csv='Police_Stations_-_Map.csv',folder='Simulation_Data')
    Gs = ox.utils_graph.get_largest_component(G, strongly=True)
    PC_node,Police_officers=sim.create_pc_node(PS_data,G,Gs,cr_lat,cr_long)
    crime_node,crime_geo=sim.get_nearest_crime_node(crime_data,Gs)
   
    time_min,ser_min,num_units=sim.create_time_sequence(crime_data)
    Crime = [time_min,crime_node,ser_min,crime_geo,num_units]
    
    df_demand=pd.DataFrame({'time':time_min,'num_units':num_units}).groupby('time').sum()
    officer_demand_tot={'Time(min)':df_demand.index.tolist(),'#units':df_demand['num_units'].tolist()}
    
    polys=sim.create_polys(PS_data,Gs, shp='Chicago_PB.shp',folder='Simulation_Data')
    ndk=sim.set_ndk(Gs,PC_node)
    
    #percent_operating=1.0
    for percent_operating in percent_operating_list:
        RT,Officers_maxed_out,officer_demand_district=sim.eva_resp_time(PC_node,Crime,percent_operating,ndk,polys,Gs)  
        res_time = sim.column(RT, 0)
        data_dict['BAU'+'_'+csv[:-13]+'_'+str(percent_operating)]=[np.mean(res_time),np.median(res_time),np.max(res_time),Officers_maxed_out,res_time,officer_demand_tot,officer_demand_district]
        
def create_df(data_dict):
    df=pd.DataFrame(data_dict).T
    df.columns=['Mean','Median','Max','Officers Maxed','RTs','Tot. Officer Demand','District Officer Demand']
    df['percentage']=[float(i[-3:]) for i in df.index]
    df['Responder']=[(i[:3]=='non')*1 for i in df.index]
    df['Responder']=df['Responder'].map({1:'Alternative',0:'Police'})#this takes into account BAU#round RTs
    df['Grouping']=[a+b for a, b in zip([('Violent' in i)*1 for i in df.index], [('BAU' in i)*2 for i in df.index])]
    df['Grouping']=df['Grouping'].map({2:'BAU',1:'Violent',0:'Index'})

    return df

In [6]:
#run_BAU(csv='all_data_0.25_1_timestamp.csv',percent_operating_list=[0.30])

In [7]:
#run_daily_profile(csv='all_data_0.25_1_timestamp.csv',grouping='Index',percent_operating_list=[0.9] )

In [8]:
arr = os.listdir('Chicago_Data/Daily_Profiles')
count=0
for csv in arr:
#for csv in ['all_data_0.25_1_timestamp.csv']:
    print(csv,round((count/len(arr))*100,1),'%')
    run_daily_profile(csv,'Index',percent_operating_list)
    run_daily_profile(csv,'Violent',percent_operating_list)
    #need to create a safety when there are no officers available 
    run_BAU(csv,percent_operating_list)
    count+=1

fall_0.5_0_timestamp.csv 0.0 %
Groups are Index and Non-index
Day Evaluating:  2014-12-03 00:00:00
Day Evaluating:  2014-12-03 00:00:00
Groups are Violent and Non-violent
Day Evaluating:  2014-12-03 00:00:00
Day Evaluating:  2014-12-03 00:00:00
BAU: No Groupings
Day Evaluating:  2014-12-03 00:00:00
summer_worst_case_0_timestamp.csv 3.6 %
Groups are Index and Non-index
Day Evaluating:  2014-08-01 00:00:00
Day Evaluating:  2014-08-01 00:00:00
Groups are Violent and Non-violent
Day Evaluating:  2014-08-01 00:00:00
Day Evaluating:  2014-08-01 00:00:00
BAU: No Groupings
Day Evaluating:  2014-08-01 00:00:00
spring_worst_case_0_timestamp.csv 7.1 %
Groups are Index and Non-index
Day Evaluating:  2014-06-01 00:00:00
Day Evaluating:  2014-06-01 00:00:00
Groups are Violent and Non-violent
Day Evaluating:  2014-06-01 00:00:00
Day Evaluating:  2014-06-01 00:00:00
BAU: No Groupings
Day Evaluating:  2014-06-01 00:00:00
winter_0.5_1_timestamp.csv 10.7 %
Groups are Index and Non-index
Day Evaluating:  

KeyboardInterrupt: 

In [9]:
df=create_df(data_dict)
df.to_csv('Chicago_Data/RT_Data_part1.csv')
df.head()

,Mean,Median,Max,Officers Maxed,RTs,Tot. Officer Demand,District Officer Demand,percentage,Responder,Grouping
Index_fall_0.5_0__0.1,58.638708,34.006741,391.684303,512,"[4.031186014142444, 1.0585106316875244, 1.2173...","{'Time(min)': [1.0, 5.0, 30.0, 39.0, 47.0, 51....","{'17': [(1.0, 2), (95.0, 2), (180.0, 2), (310....",0.1,Police,Index
nonIndex_fall_0.5_0__0.9,3.189469,2.979565,9.133439,0,"[5.014656008052992, 4.897465420607205, 5.66690...","{'Time(min)': [0.0, 1.0, 5.0, 30.0, 35.0, 45.0...","{'17': [(1.0, 2), (120.0, 1), (590.0, 1), (720...",0.9,Alternative,Index
Index_fall_0.5_0__0.2,14.59091,4.364969,118.667287,282,"[4.031186014142444, 4.031186014142444, 1.05851...","{'Time(min)': [1.0, 5.0, 30.0, 39.0, 47.0, 51....","{'17': [(1.0, 2), (95.0, 2), (180.0, 1), (310....",0.2,Police,Index
nonIndex_fall_0.5_0__0.8,3.189469,2.979565,9.133439,0,"[5.014656008052992, 4.897465420607205, 5.66690...","{'Time(min)': [0.0, 1.0, 5.0, 30.0, 35.0, 45.0...","{'17': [(1.0, 2), (120.0, 1), (590.0, 1), (720...",0.8,Alternative,Index
Index_fall_0.5_0__0.3,6.652795,3.220244,82.582513,126,"[4.031186014142444, 4.031186014142444, 2.77464...","{'Time(min)': [1.0, 5.0, 30.0, 39.0, 47.0, 51....","{'17': [(1.0, 2), (95.0, 2), (180.0, 0), (310....",0.3,Police,Index


# Below we have the process step-by-step for reference/de-bugging

### Format Crime Data

In [4]:

"""CSV Options
daily_profile='all_data_0.5_0_timestamp.csv'
daily_profile='fall_0.5_0_timestamp.csv'
daily_profile='spring_0.5_0_timestamp.csv'
daily_profile='winter_0.5_0_timestamp.csv'
daily_profile='summer_0.5_0_timestamp.csv'

daily_profile='all_data_worst_case_0_timestamp.csv'
daily_profile='spring_worst_case_0_timestamp.csv'
daily_profile='summer_worst_case_0_timestamp.csv'
daily_profile='winter_worst_case_0_timestamp.csv'
daily_profile='fall_worst_case_0_timestamp.csv

grouping options:
grouping='Violent'
grouping='Index'
'"""
crime_data,polrep_crime_data,altrep_crime_data=sim.create_crime_dfs(csv='all_data_0.25_0_timestamp.csv',grouping='Index')


Groups are Index and Non-index


In [34]:
pd.to_datetime(crime_data['Timestamp'])

0     2014-02-28 00:00:00
1     2014-02-28 00:00:00
2     2014-02-28 00:00:00
3     2014-02-28 00:00:00
4     2014-02-28 00:00:00
              ...        
693   2014-02-28 23:45:00
694   2014-02-28 23:50:00
695   2014-02-28 23:50:00
696   2014-02-28 23:50:00
697   2014-02-28 23:54:00
Name: Timestamp, Length: 698, dtype: datetime64[ns]

## Importing Police Location Data (Using that as location for police for basic simulation)

In [5]:
PS_data,cr_lat,cr_long=sim.create_police_station_df(csv='Police_Stations_-_Map.csv',folder='Simulation_Data')
Gs = ox.utils_graph.get_largest_component(G, strongly=True)
PC_node,Police_officers=sim.create_pc_node(PS_data,G,Gs,cr_lat,cr_long)

In [19]:
PC_node

[704990298,
 261199941,
 261121479,
 305905861,
 456155217,
 261160897,
 349885250,
 2040146197,
 261253127,
 1446789454,
 3193134832,
 261113958,
 261290982,
 261169883,
 261109512,
 305527080,
 344365298,
 1309327014,
 256181100,
 261191895,
 261176804,
 305911655,
 250278999]

## Evaluating Response time based on the nearest police location to the crime

## Initialize varibales : no of police associated with police location, no of police needed based on crime, 

In [6]:
polrep_crime_node,polrep_crime_geo=sim.get_nearest_crime_node(polrep_crime_data,Gs)
altrep_crime_node,altrep_crime_geo=sim.get_nearest_crime_node(altrep_crime_data,Gs)

In [7]:
polrep_time_min,polrep_ser_min,num_units=sim.create_time_sequence(polrep_crime_data)
polrep_Crime = [polrep_time_min,polrep_crime_node,polrep_ser_min,polrep_crime_geo,num_units]

Day Evaluating:  2014-02-28 00:00:00


In [8]:
altrep_time_min,altrep_ser_min,num_units=sim.create_time_sequence(altrep_crime_data)
alt_rep_Crime = [altrep_time_min,altrep_crime_node,altrep_ser_min,altrep_crime_geo,num_units]

Day Evaluating:  2014-02-28 00:00:00


df_test=pd.DataFrame({'time':altrep_time_min,'num_units':num_units}).groupby('time').sum()
df_test

In [23]:
officer_demand_tot={'Time(min)':df_test.index.tolist(),'#units':df_test['num_units'].tolist()}

## Police Beats

In [10]:
warnings.filterwarnings("ignore")
polys=sim.create_polys(PS_data,Gs, shp='Chicago_PB.shp',folder='Simulation_Data')

In [15]:
percent_operating=0.8
ndk=sim.set_ndk(Gs,PC_node)
Resp_time,Officers_maxed_out,officer_demand_district=sim.eva_resp_time(PC_node,polrep_Crime,percent_operating,ndk,polys,Gs)

police node 261290982
num_units top 3
PC_stalk [261290982, 0, 0, [], [], [], [], [], []]
#off 8
num_units bottom 2
num_units top 2
PC_stalk [261290982, 1, 0, [0.0], [261157751], [29.543646463767757], [], [], []]
#off 8
num_units bottom 1
num_units top 1
PC_stalk [261290982, 2, 0, [0.0, 0.0], [261157751, 261157751], [29.543646463767757, 29.543646463767757], [], [], []]
#off 8
num_units bottom 0
police node 261113958
num_units top 2
PC_stalk [261113958, 0, 0, [], [], [], [], [], []]
#off 13
num_units bottom 1
num_units top 1
PC_stalk [261113958, 1, 0, [0.0], [261271616], [20.496959623199572], [], [], []]
#off 13
num_units bottom 0
police node 3193134832
num_units top 2
PC_stalk [3193134832, 0, 0, [], [], [], [], [], []]
#off 12
num_units bottom 1
num_units top 1
PC_stalk [3193134832, 1, 0, [0.0], [261143318], [24.13274323635776], [], [], []]
#off 12
num_units bottom 0
police node 261199941
num_units top 2
PC_stalk [261199941, 0, 0, [], [], [], [], [], []]
#off 12
num_units bottom 1
num_u

{'17': [(555.0, 3),
  (675.0, 2),
  (885.0, 2),
  (885.0, 4),
  (1255.0, 2),
  (1260.0, 0),
  (1350.0, 2),
  (1380.0, 3),
  (1382.0, 6)],
 '20': [(1020.0, 3), (1020.0, 5), (1200.0, 2), (1230.0, 5), (1260.0, 2)],
 '16': [(0.0, 3),
  (120.0, 3),
  (480.0, 2),
  (680.0, 2),
  (770.0, 2),
  (870.0, 2),
  (990.0, 2),
  (1050.0, 3),
  (1200.0, 3),
  (1260.0, 2),
  (1320.0, 2)],
 '19': [(150.0, 3),
  (510.0, 3),
  (540.0, 2),
  (860.0, 2),
  (870.0, 3),
  (900.0, 2),
  (910.0, 4),
  (960.0, 2),
  (971.0, 5),
  (1020.0, 2),
  (1020.0, 5),
  (1080.0, 2),
  (1350.0, 3),
  (1365.0, 5)],
 '25': [(415.0, 2),
  (465.0, 3),
  (540.0, 3),
  (660.0, 3),
  (830.0, 3),
  (840.0, 5),
  (980.0, 2),
  (1125.0, 2),
  (1200.0, 2),
  (1320.0, 3),
  (1410.0, 2)],
 '14': [(0.0, 3),
  (0.0, 6),
  (120.0, 3),
  (450.0, 3),
  (537.0, 2),
  (660.0, 2),
  (780.0, 2),
  (840.0, 3),
  (915.0, 2),
  (930.0, 2),
  (960.0, 4),
  (1020.0, 3),
  (1050.0, 2),
  (1170.0, 2),
  (1191.0, 4),
  (1220.0, 2),
  (1370.0, 3)],
 '22'

In [20]:
#for later reference
officer_demand #add as column
res = [[ i for i, j in officer_demand['17'],
       [ j for i, j in officer_demand['17'] ]]
      

In [21]:
res

[[555.0, 675.0, 885.0, 885.0, 1255.0, 1260.0, 1350.0, 1380.0, 1382.0],
 [3, 2, 2, 4, 2, 0, 2, 3, 6]]